# Install Necessary Packages


In [ ]:
pip install torch torchvision numpy opencv-python

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import os
import json
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageDraw

# Dataset Class Definition
This class handles loading of black and pink hoodie images and their corresponding annotations.

In [ ]:
class HoodieDataset(Dataset):
    def __init__(self, black_image_dir, black_annotation_dir, pink_image_dir, pink_annotation_dir, transform=None):
        self.black_image_dir = black_image_dir
        self.black_annotation_dir = black_annotation_dir
        self.pink_image_dir = pink_image_dir
        self.pink_annotation_dir = pink_annotation_dir

        self.black_images = [(os.path.join(black_image_dir, f), os.path.join(black_annotation_dir, os.path.splitext(f)[0] + 'mask.json'))
                             for f in os.listdir(black_image_dir) if os.path.isfile(os.path.join(black_image_dir, f))]
        self.pink_images = [(os.path.join(pink_image_dir, f), os.path.join(pink_annotation_dir, os.path.splitext(f)[0] + 'mask.json'))
                            for f in os.listdir(pink_image_dir) if os.path.isfile(os.path.join(pink_image_dir, f))]

        self.images = self.black_images + self.pink_images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path, annotation_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        annotations = self.load_annotations(annotation_path)
        mask = self.create_mask(image.size, annotations)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        color_label = 0 if image_path.startswith(self.black_image_dir) else 1
        color_label = torch.tensor(color_label, dtype=torch.float32).unsqueeze(0)

        return image, mask, color_label

    def load_annotations(self, annotation_file):
        with open(annotation_file, 'r') as f:
            annotations = json.load(f)
        return annotations

    def create_mask(self, image_size, annotations):
        mask = Image.new('L', image_size, 0)
        for shape in annotations['shapes']:
            points = shape['points']
            label = shape['label']
            color = self.get_color_for_label(label)
            polygon = [(int(x), int(y)) for x, y in points]
            ImageDraw.Draw(mask).polygon(polygon, outline=color, fill=color)
        return mask

    def get_color_for_label(self, label):
        color_map = {
            'Hood': 1,
            'Logo': 2,
            'Sleeves': 3,
            'Cuffs': 4,
            'Waistband': 5,
            'Front Bodies': 6
        }
        return color_map.get(label, 0)

# Define the Generator and Discriminator
These classes define the architectures of the generator and discriminator.

In [ ]:
# Define the Generator and Discriminator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(101, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input, color_label):
        # Concatenate color_label with the input noise
        combined_input = torch.cat((input, color_label), 1)
        return self.main(combined_input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Initialize models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
netG = Generator().to(device)
netD = Discriminator().to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Load the dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = HoodieDataset('/content/drive/MyDrive/data /black hoodies/', '/content/drive/MyDrive/Hood_Masked/black hoodies/', '/content/drive/MyDrive/data /pink hoodies/', '/content/drive/MyDrive/Hood_Masked/pink hoodies/', transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
  try:
    for i, (images, masks, color_labels) in enumerate(dataloader, 0):
        # Update Discriminator
        netD.zero_grad()
        real_images = images.to(device)
        batch_size = real_images.size(0)
        labels = torch.full((batch_size,), 1, device=device)
        output = netD(real_images)
        lossD_real = criterion(output, labels)
        lossD_real.backward()

        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        fake_images = netG(noise, color_labels.to(device))
        labels.fill_(0)
        output = netD(fake_images.detach())
        lossD_fake = criterion(output, labels)
        lossD_fake.backward()
        optimizerD.step()

        # Update Generator
        netG.zero_grad()
        labels.fill_(1)
        output = netD(fake_images)
        lossG = criterion(output, labels)
        lossG.backward()
        optimizerG.step()

        print(f'[{epoch}/{num_epochs}][{i}/{len(dataloader)}] Loss_D: {lossD_real.item() + lossD_fake.item()} Loss_G: {lossG.item()}')
  except Exception as e:
    print(e)
    # Save the model checkpoints
    torch.save(netG.state_dict(), 'generator.pth')
    torch.save(netD.state_dict(), 'discriminator.pth')


[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/pink hoodies/7415c5855edcb9c8ab9a087c6ffdf94emask.json'
[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/black hoodies/a9c8d66046fafcff07f9d582b5df0926mask.json'
[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/pink hoodies/4bd19f05f463bbc66813be4144cdfc27mask.json'
[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/pink hoodies/4bd19f05f463bbc66813be4144cdfc27mask.json'
[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/pink hoodies/ec40c84ee4d1f0777a9f190d7f18e857mask.json'
[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/pink hoodies/4bd19f05f463bbc66813be4144cdfc27mask.json'
[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/pink hoodies/4bd19f05f463bbc66813be4144cdfc27mask.json'
[Errno 2] No such file or directory: '/content/drive/MyDrive/Hood_Masked/black hoodies/0b5e8cf05